<a href="https://colab.research.google.com/github/YamgoKES/Structured-Data-Machine-Learning-Modelling/blob/main/notebook/mini_proj_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

import matplotlib.font_manager as fm  # 폰트 관리

!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
'''with open('/content/drive/MyDrive/EST_soft 부트캠프/프로젝트/csv파일모음/new_gas.csv', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']'''

# 추정된 인코딩으로 파일 읽기 ....
df = pd.read_csv('/content/new_gas.csv')

In [ ]:
df.head(3)

,상호,상표,셀프여부,휘발유,경유,법정동명,month,공시지가(원/㎡),평균 공시지가(원/㎡),Dubai,14일 전 환율
0,(유)동하석유 힐탑셀프주유소,SK에너지,셀프,1737,1697,논현동,1,9670000.0,12473509.2,76.17,1294.7
1,(유)동하석유 힐탑셀프주유소,SK에너지,셀프,1737,1697,논현동,1,9670000.0,12473509.2,76.83,1298.7
2,(유)동하석유 힐탑셀프주유소,SK에너지,셀프,1737,1697,논현동,1,9670000.0,12473509.2,79.06,1304.7


칼럼 설명 <br>
상호 : 대리점 이름 <br>
상표 : 브랜드 이름 <br>
휘발유/경유 : 단위 리터당 가격(원) <br>
공시지가 : 법정동명 기준으로의 중앙값 <br>
평균 공시지가  : 말 그대로 법정동명기준으로의 평균 공시지가 <br>
Dubai : 두바이유 (2주전 데이터) <br>
14일 전 환율 : 반영시기가 2주전 정도이기 때문에 적용함 <br>

 독립변수  : 공시지가(0.45) , 셀프 , 법정동명 , month , Dubai, 14일전 환율 <br>
 종속변수 : 휘발유 가격

In [ ]:
############ 전처리전 시각화 보기 ########################

In [ ]:
############### 전처리 ################## 상관관계 ######################

In [ ]:
df['상표']=df['상표'].replace('알뜰(ex)', '알뜰주유소')

In [ ]:
# 휘발유가 0인값을 그 달의 평균값으로 채워넣기

zero_values=df['휘발유']==0 #df['휘발유'] != 0

month_avg=df[df['휘발유'] != 0].groupby('month')['휘발유'].mean() #0값을 제외한 월별 평균값

for idx in df[zero_values].index :
    month = df.loc[idx, 'month']
    df.loc[idx, '휘발유'] = month_avg[month]

<ipython-input-8-e4459580ca28>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1643.1348851709447' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, '휘발유'] = month_avg[month]


In [ ]:
df[df==0].count()

,0
상호,0
상표,0
셀프여부,0
휘발유,0
경유,51
법정동명,0
month,0
공시지가(원/㎡),0
평균 공시지가(원/㎡),0
Dubai,0


In [ ]:
#모델링 과정 시작

In [ ]:
#데이터 x, y 나누기

x=df.drop(columns=['상호','휘발유','경유', '평균 공시지가(원/㎡)']) # 독립변수 설정
y=df['휘발유']

In [ ]:
#############

In [ ]:
x

,상표,셀프여부,법정동명,month,공시지가(원/㎡),Dubai,14일 전 환율
0,SK에너지,셀프,논현동,1,9670000.0,76.17,1294.7
1,SK에너지,셀프,논현동,1,9670000.0,76.83,1298.7
2,SK에너지,셀프,논현동,1,9670000.0,79.06,1304.7
3,SK에너지,셀프,논현동,1,9670000.0,78.84,1300.2
4,SK에너지,셀프,논현동,1,9670000.0,79.12,1303.8
...,...,...,...,...,...,...,...
117722,GS칼텍스,셀프,망우동,12,2913000.0,73.68,1431.4
117723,GS칼텍스,셀프,망우동,12,2913000.0,73.47,1431.5
117724,GS칼텍스,셀프,망우동,12,2913000.0,73.47,1431.5
117725,GS칼텍스,셀프,망우동,12,2913000.0,74.14,1432.7


In [ ]:
#독립변수의 범주형 변수 원핫인코딩
x=pd.get_dummies(x, columns=['상표','셀프여부', '법정동명' ,'month'],drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#scaling standard scaling 적용예정
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
num_col=['공시지가(원/㎡)','Dubai', '14일 전 환율']
x_train[num_col]=scaler.fit_transform(x_train[num_col])
x_test[num_col] = scaler.transform(x_test[num_col])

#확인
print(x_train[num_col].head())

        공시지가(원/㎡)     Dubai  14일 전 환율
36611    1.283893  1.220348 -0.352830
85328   -0.262033 -1.072210  2.470971
53580   -0.281077  0.807800  0.689248
37934   -0.547702  0.323670 -0.773606
107682   3.675234 -0.049318  0.889774


##1. RandomForest Regression

In [ ]:
## 1. RandomForest Regression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest=RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=4
    )

In [ ]:
random_forest.fit(x_train, y_train)

In [ ]:
pred_random_forest = random_forest.predict(x_test)

##Random Forest 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

test_mse = mean_squared_error(y_test, pred_random_forest) # mse
test_r2 = r2_score(y_test, pred_random_forest) # r2
test_mae = mean_absolute_error(y_test, pred_random_forest) # mae


print(f'MAE : {test_mae : .2f}')
print(f'MSE: :{test_mse : .2f}')
print(f'R2 : {test_r2 : .3f}')


In [ ]:
y_train_pred = random_forest.predict(x_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f'MSE: :{train_mse : .2f}')
print(f'R2 : {train_r2 : .3f}')

print(f'MSE차이 : {test_mse-train_mse :.2f}')
print(f'R2 차이 : {train_r2-test_r2 : .3f}')

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression=LinearRegression()
linear_regression.fit(x_train,y_train)


In [ ]:
pred_linear_regression=linear_regression.predict(x_test)

## 평가

In [ ]:
test_mse_lr= mean_squared_error(y_test, pred_linear_regression)
test_mae_lr= mean_absolute_error(y_test, pred_linear_regression)
test_r2_lr=r2_score(y_test,pred_linear_regression)

print(f'MAE : {test_mae_lr : .2f}')
print(f'MSE: :{test_mse_lr : .2f}')
print(f'R2 : {test_r2_lr : .3f}')

y_train_pred_lr = linear_regression.predict(x_train)
train_mse_lr= mean_squared_error(y_train, y_train_pred_lr)
train_r2_lr=r2_score(y_train,y_train_pred_lr)

print(f'MSE: :{train_mse_lr : .2f}')
print(f'R2 : {train_r2_lr : .3f}')


print(f'MSE차이 : {test_mse_lr-train_mse_lr :.2f}')
print(f'R2 차이 : {train_r2_lr-test_r2_lr : .3f}')

## XGBoost
Gradient Boosting 기반으로 동작하며, XGBRegressor를 활용해 모델링  
하이퍼파라미터-트리 개수(n_estimators), 최대 깊이(max_depth), 학습률(learning_rate)

In [ ]:
# 1. XGBoost 모델 초기화
from xgboost import XGBRegressor

xgboost = XGBRegressor(
    n_estimators=300,      # 생성할 트리의 개수
    max_depth=20,          # 트리의 최대 깊이
    learning_rate=0.1,     # 학습률
    random_state=42        # 재현성을 위한 난수 고정
)

In [ ]:
# 2. 모델 학습
xgboost.fit(x_train, y_train)

In [ ]:
# 3. 예측
pred_xgboost = xgboost.predict(x_test)

## 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# 4. 평가
test_mse_xgboost = mean_squared_error(y_test, pred_xgboost)
test_mae_xgboost = mean_absolute_error(y_test, pred_xgboost)
test_r2_xgboost = r2_score(y_test, pred_xgboost)

print(f'XGBoost MAE: {test_mae_xgboost:.2f}')
print(f'XGBoost MSE: {test_mse_xgboost:.2f}')
print(f'XGBoost R2: {test_r2_xgboost:.3f}')

In [ ]:
y_train_pred_xgboost = xgboost.predict(x_train)
train_mse_xgboost = mean_squared_error(y_train, y_train_pred_xgboost)
train_r2_xgboost = r2_score(y_train, y_train_pred_xgboost)

print(f'XGBoost Train MSE: {train_mse_xgboost:.2f}')
print(f'XGBoost Train R2: {train_r2_xgboost:.3f}')
print(f'XGBoost MSE Diff: {test_mse_xgboost - train_mse_xgboost:.2f}')
print(f'XGBoost R2 Diff: {train_r2_xgboost - test_r2_xgboost:.3f}')

## KNN 모델링

In [ ]:
# 1. KNN 모델 초기화
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(
    n_neighbors=3,         # 고려할 이웃의 개수
    weights='distance',     # 이웃 간 동일한 가중치
    metric='minkowski',    # 거리 측정 기준 (디폴트: 유클리드 거리)
    p=1                    # k=1이면 맨해튼
)

In [ ]:
# 2. 모델 학습
knn.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=3, p=1, weights='distance')

In [ ]:
# 3. 예측
pred_knn = knn.predict(x_test)

## 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 4. 평가
test_mse_knn = mean_squared_error(y_test, pred_knn)
test_mae_knn = mean_absolute_error(y_test, pred_knn)
test_r2_knn = r2_score(y_test, pred_knn)

print(f'KNN MAE: {test_mae_knn:.2f}')
print(f'KNN MSE: {test_mse_knn:.2f}')
print(f'KNN R2: {test_r2_knn:.3f}')

KNN MAE: 23.27
KNN MSE: 4955.60
KNN R2: 0.845


In [ ]:
y_train_pred_knn = knn.predict(x_train)
train_mse_knn = mean_squared_error(y_train, y_train_pred_knn)
train_r2_knn = r2_score(y_train, y_train_pred_knn)

print(f'KNN Train MSE: {train_mse_knn:.2f}')
print(f'KNN Train R2: {train_r2_knn:.3f}')
print(f'KNN MSE Diff: {test_mse_knn - train_mse_knn:.2f}')
print(f'KNN R2 Diff: {train_r2_knn - test_r2_knn:.3f}')

KNN Train MSE: 1202.77
KNN Train R2: 0.963
KNN MSE Diff: 3752.82
KNN R2 Diff: 0.118


In [ ]:
df

In [ ]:
x

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
# 하이퍼파라미터 조합
param_grid = [
    {'n_neighbors': 3, 'weights': 'uniform', 'p': 1, 'metric': 'minkowski'},
    {'n_neighbors': 5, 'weights': 'distance', 'p': 2, 'metric': 'minkowski'},
    {'n_neighbors': 10, 'weights': 'uniform', 'p': 1, 'metric': 'manhattan'},

]

# 반복 실행 및 평가 (기존 코드 동일)
results = pd.DataFrame(columns=['n_neighbors', 'weights', 'p', 'metric',
                                'MAE', 'MSE', 'R^2', 'mse-train_mse', 'train_r2-r2'])

for params in param_grid:
    knn = KNeighborsRegressor(
        n_neighbors=params['n_neighbors'],
        weights=params['weights'],
        metric=params['metric'],
        p=params['p']
    )
    knn.fit(x_train, y_train)
    pred_test = knn.predict(x_test)
    mae = mean_absolute_error(y_test, pred_test)
    mse = mean_squared_error(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    pred_train = knn.predict(x_train)
    train_mse = mean_squared_error(y_train, pred_train)
    train_r2 = r2_score(y_train, pred_train)
    mse_diff = train_mse - mse
    r2_diff = train_r2 - r2
    new_row = pd.DataFrame([{
        'n_neighbors': params['n_neighbors'],
        'weights': params['weights'],
        'p': params['p'],
        'metric': params['metric'],
        'MAE': mae,
        'MSE': mse,
        'R^2': r2,
        'mse-train_mse': mse_diff,
        'train_r2-r2': r2_diff
    }])
    results = pd.concat([results, new_row], ignore_index=True)

# 결과 출력
print(results)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
# 하이퍼파라미터 조합
param_grid = [
    {'n_neighbors': 3, 'weights': 'uniform', 'p': 1, 'metric': 'euclidean'},
    {'n_neighbors': 5, 'weights': 'uniform', 'p': 2, 'metric': 'minkowski'},
    {'n_neighbors': 10, 'weights': 'distance', 'p': 1, 'metric': 'manhattan'},

]

# 반복 실행 및 평가 (기존 코드 동일)
results = pd.DataFrame(columns=['n_neighbors', 'weights', 'p', 'metric',
                                'MAE', 'MSE', 'R^2', 'mse-train_mse', 'train_r2-r2'])

for params in param_grid:
    knn = KNeighborsRegressor(
        n_neighbors=params['n_neighbors'],
        weights=params['weights'],
        metric=params['metric'],
        p=params['p']
    )
    knn.fit(x_train, y_train)
    pred_test = knn.predict(x_test)
    mae = mean_absolute_error(y_test, pred_test)
    mse = mean_squared_error(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    pred_train = knn.predict(x_train)
    train_mse = mean_squared_error(y_train, pred_train)
    train_r2 = r2_score(y_train, pred_train)
    mse_diff = train_mse - mse
    r2_diff = train_r2 - r2
    new_row = pd.DataFrame([{
        'n_neighbors': params['n_neighbors'],
        'weights': params['weights'],
        'p': params['p'],
        'metric': params['metric'],
        'MAE': mae,
        'MSE': mse,
        'R^2': r2,
        'mse-train_mse': mse_diff,
        'train_r2-r2': r2_diff
    }])
    results = pd.concat([results, new_row], ignore_index=True)

# 결과 출력
print(results)


In [ ]:
results.to_excel('result2.xlsx', index=False)  # index=False는 인덱스를 저장하지 않도록 설정